In [1]:
# Libraries
import numpy as np
import gym
import random

# Step 1: Create the environment
* Creating the Taxi environment
* OpenAI is a library **composed of many environments that can be used to train an agent**

In [2]:
env = gym.make('Taxi-v2')
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Step 2: Create the Q-table and initialize it
* The Q-matrix consists of a m x n matrix. m represents the n umber of states, and n represents the number of actions per state
* OpenAI Gym provides us a way to get these parameters, as follows:

In [3]:
action_size = env.action_space.n
print("Action size: ", action_size)

state_size = env.observation_space.n
print("State space: ", state_size)

Action size:  6
State space:  500


In [4]:
# Construct the Q-matrix:
q_table = np.zeros((state_size, action_size))
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# Step 3: Create the hyperparameters

In [5]:
total_episodes = 50000     # Total episodes to train our algorithm
total_test_episodes = 10  # Total episodes to test our algorithm
max_steps = 99             # Max steps per episode

learning_rate = 0.7        # Learning rate
gamma = 0.618              # Discount rate

# Exploration parameters
epsilon = 1.0              # Exploration rate
max_epsilon = 1.0          # Exploration probability at start
min_epsilon = 0.01         # Minimum exploration probability
decay_rate = 0.01          # Exponential decay rate for exploration

# Step 4: The Q learn algorithm
Now we implement the Q learn algorithm:
1. Initialize Q-values ($Q(s,a)$) arbitrarily for all (state, action) pairs.
2. While alive or until episode is stopped:...
    1. Choos an action ($a$) in the current world state ($s$) based on the current Q-value estimates ($Q(s,\cdot)$).
    2. Take the action ($a$) and observe the outcome state ($s\prime$) and reward ($r$)
    3. Update $Q(s,a) = Q(s,a) + \alpha\cdot[r+\gamma \cdot \max_{a\prime}Q(s\prime,a\prime)-Q(s,a)]$ (Bellman equation)

In [6]:
# For each episode, run until 'life ends' or 'learning is stopped':
for episode in range(total_episodes):
    # Reset environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        # Choose the action (a) in the current world state (s)
        ## First randomize a number
        exp_exp_tradeoff = random.uniform(0,1)  # Decide between exploit/explore
        
        ## If number > epsilon, then choose exploitation instead of exploration (taking the biggest reachable Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(q_table[state,:])  # Returns the Q-index for the maximum value in a given state
            
        ## Else: do a random choice - a.k.a. exploration
        else:
            action = env.action_space.sample()
            
        # Take the action (a) and observe the outcome state (s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Update Q(s,a) using the Bellman equation
        q_table[state, action] = (q_table[state, action] 
                                  + learning_rate * (reward + gamma * 
                                                      np.max(q_table[new_state, :]) - q_table[state, action]))
        
        # Update state
        state = new_state
        
        # If done: finish episode
        if done:
            break
            
    episode += 1
    
    # Reduce epsilon (to ensure less exploration and more explotation over time)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episode)

# Step 5: Use our Q-table to play Taxi
* After 'total_episodes' episodes, the modifyed Q-table can be used as a brute-forced 'cheat sheet' to play taxi
* By running this cell, you can see the agent play the game

In [7]:
env.reset()
rewards = []
ep_frames = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    ep_frames.append([])
    
    #print('**************************************************')
    #print('EPISODE {}/{}'.format(episode, total_test_episodes))
    
    for step in range(max_steps):
        # Uncomment if you want to see the agent's gameplay
        #env.render()
        
        # Take the action (index) that has the maximum expected future reward, given a state
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        ep_frames[-1].append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward,
            'episode': episode
        })
        
        if done:
            rewards.append(total_rewards)
            print('Ep.#{}'.format(episode))
            print('Score: ', total_rewards)
            print('Steps: ', step, '\n')
            break
            
        state = new_state
                
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Ep.#0
Score:  8
Steps:  12 

Ep.#1
Score:  5
Steps:  15 

Ep.#2
Score:  7
Steps:  13 

Ep.#3
Score:  5
Steps:  15 

Ep.#4
Score:  9
Steps:  11 

Ep.#5
Score:  3
Steps:  17 

Ep.#6
Score:  10
Steps:  10 

Ep.#7
Score:  7
Steps:  13 

Ep.#8
Score:  11
Steps:  9 

Ep.#9
Score:  6
Steps:  14 

Score over time: 7.1


In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(ep_frames):
    for ep_i, frames in enumerate(ep_frames):
        acc_reward = 0
        for i, frame in enumerate(frames):
            clear_output(wait=True)
            print(frame['frame'].getvalue())
            print(f"Episode: {frame['episode']}")
            print(f"Timestep: {i + 1}")
            print(f"State: {frame['state']}")
            print(f"Action: {frame['action']}")
            print(f"Reward: {frame['reward']}")
            acc_reward += frame['reward']
            print(f"Acc.Reward: {acc_reward}")
            tanh = lambda x, x0=-1, xN=1, y0=-1, yN=1: (yN-y0)/2 * np.tanh(2*np.pi*((x-x0)/(xN-x0)) - np.pi) + y0 + (yN-y0)/2  # tanh that goes from 0 to 1
            sleep(tanh(ep_i, 0, len(ep_frames), 1, 0.1))
        
print_frames(ep_frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 9
Timestep: 15
State: 479
Action: 5
Reward: 20
Acc.Reward: 6


In [9]:
import matplotlib.pyplot as plt

func = lambda x, x0=-1, xN=1, y0=-1, yN=1: (yN-y0)/2 * np.tanh(2*np.pi*((x-x0)/(xN-x0)) - np.pi) + y0 + (yN-y0)/2  # tanh that goes from 0 to 1
xpts = np.linspace(0, len(ep_frames), 100)

plt.plot(xpts, func(xpts, 0, len(ep_frames), 1, 0.1))
plt.axhline(y=0.1, color='r', linestyle='--')

plt.title("Scaled tanh()-function")
plt.xlabel("Episode")
plt.ylabel("Seconds per frame [1/fps]")
plt.ylim([0,1])
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
len(ep_frames)

10